Problem 1

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from numpy.linalg import inv

In [3]:

def generate_c(num_c_sets):
    coef_list = []
    for i in range(num_c_sets):
        coef = np.random.rand(3)
        coef_list.append(coef)
    return coef_list

NUM_C_SETS = 1
COEF_LIST = generate_c(NUM_C_SETS)
# 3 different choices of N
N_LIST = [10]#[5,10,20, 30, 40]
NOISE_INTENSITY_LIST = [0.1, 0.5, 3]

# generate N predictions yi
def predicted_function(x_array, coef):
    c1 = coef[0]
    c2 = coef[1]
    c3 = coef[2]
    return c1 + c2 * np.exp(c3 * x_array)

# randomly sample N xi
def random_x(n):
    return np.linspace(0,1,n)

# add noise to calculated y_array to get simulated real yi_array
def add_noise(y_array, noise_intensivity):
    noise = noise_intensivity * np.random.randn(len(y_array))
    yi_noise = y_array + noise
    return yi_noise


def y_and_fit(x_array, y_noise, c_list, coef, n, noise_intensity):
    x_even = np.linspace(0,1, 100)
    y_pred = predicted_function(x_even, c_list[-1])
    initial_y = predicted_function(x_even, coef)
    plt.scatter(x_array, y_noise)
    plt.plot(x_even, y_pred, 'b', label = 'fit after adding noise')
    plt.plot(x_even, initial_y, 'g', label = 'initial')
    plt.ylim((-3,5))
    plt.legend()
    plt.title(f'Noise intensity:{noise_intensity}')
    plt.show()
    

def see_cost_reduction(cost_list):
    iteration = range(1, len(cost_list))
    plt.scatter(iteration, cost_list[1:], s= 10)
    plt.title('cost as iteration increases')
    plt.xlabel('iteration')
    plt.ylabel('cost')




def jacobian(coef, x_array):
    c1, c2, c3 = coef[0], coef[1], coef[2]
    n = len(x_array)
    first_col = np.ones(n)
    second_col = np.exp(c3 * x_array)
    third_col = c2 * x_array * np.exp(c3 * x_array)
    Ja = np.array([first_col,second_col, third_col]).transpose()
    return Ja
# Jacobian: n * 3

def residual(y_noise, y_array):
    residual = y_noise - y_array
    return residual

def solve_v(Ja, residual):
    A = np.matmul(np.transpose(Ja), Ja)
    b = np.matmul(np.transpose(Ja), residual)
    v = np.linalg.solve(A, b)
    return v

def gaussian_newton(c_initial, x_array, y_noise):
    # initiate coefficients and cost
    c_list = [c_initial]

    cost_list = [0, np.sum(np.square(y_noise - predicted_function(x_array, c_initial)))]
    i = 0
    # iterate until the reduction of cost <0.0001
    while i==1 or abs(cost_list[-1] - cost_list[-2])>0.0001:
        cost_old = cost_list[-1]
        c_old = c_list[-1]
        Ja_i = jacobian(c_old, x_array)
        residual_i = residual(y_noise, predicted_function(x_array, c_old))
        v = solve_v(Ja_i, residual_i)
        beta_power = 0
        while True:
            beta = 0.9
            c_new = c_old - (beta)**(beta_power) * v
            #print(f'y_noise{y_noise}')
            #print(f'predict:{predicted_function(x_array, c_new)}')
            cost_new = np.sum(np.square(y_noise - predicted_function(x_array, c_new)))
            if cost_new < cost_old:
                break
            beta_power +=1
        c_list.append(c_new)
        cost_list.append(cost_new)
        i +=1
    i_till_convergence = i
    return cost_list, c_list, i_till_convergence



def data_generation_and_gauss_newton():
    global COEF_LIST, N_LIST, NOISE_INTENSITY_LIST
    
    for i in range(len(COEF_LIST)):
        coef = COEF_LIST[i]
        for j in range(len(N_LIST)):
            n = N_LIST[j]
            x_array = random_x(n)
            for k in range(len(NOISE_INTENSITY_LIST)):
                noise_intensity = NOISE_INTENSITY_LIST[k]
                y_array = predicted_function(x_array, coef)
                y_noise = add_noise(y_array, noise_intensity)
                print(f'EXPERIMENT: coef:{coef}, N:{n},noise_intensity:{noise_intensity}')
                c_initial = np.array([0.7,0.9,0.8])
                cost_list, c_list, i_till_convergence = gaussian_newton(c_initial, x_array, y_noise)
                print(f'cost_list:{cost_list}')
                print(f'number of iterations until convergence:{i_till_convergence}\n')
                #y_and_fit(x_array, y_noise, c_list, coef, n, noise_intensity)
                see_cost_reduction(cost_list)
        

data_generation_and_gauss_newton()

EXPERIMENT: coef:[0.74198609 0.51915592 0.69785354], N:10,noise_intensity:0.1


Observations: The cost conveges to a minimum value with a proper initial guess of coefficients. Initial guess of coefficients matters( in this case the magnitude of c can not be too large, the choice of c should also not induce a singular jacobian matrix). The plot shows that cost convergence trend of different combinations of randomly selected coefficients generating yis, number of xi, and noise intensity. The number of iteration till convergences vary depending on the choice of coefficients generating yis, N, and noise intensity, but the value of the cost function always shrink rapidly within the first few iterations with a good initial guess of coefficients. The speed of convergence decreases as the number of iteration increases. When the randomized coefficients generating yis and N are controlled, as the intensity of noise increases, the simulated yi becomes more scattered and the optimized cost increases, which is as expected considering the formula of MMSE. However, the number of iterations to get an optimal coefficient set c is not directly influenced by the intensity of noise. Similarly, when randomized coefficients generating yis and noise intensity are controlled, the optimized cost increases as the number of xis increases, which is also as expected given the formula of MMSE. However, the number of iterations to get an optimal coefficient set c is not directly influenced by the the number of xis. 






